## Predict E-Commerce Order Cancellations with XGBoost and Jupyter Notebooks

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb

#### Read and preview data

In [3]:
order_data = pd.read_csv('OnlineRetail-clean.csv')

In [4]:
order_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,ImputedCustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom


### Preprocessing

#### Creat target dependent variable 'IsCancellation'

In [6]:
order_data['isCancellation'] = order_data['InvoiceNo'].str.startswith('C')

In [19]:
order_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,ImputedCustomerID,Country,isCancellation
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,17850,United Kingdom,False
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom,False
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,17850,United Kingdom,False
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom,False
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,17850,United Kingdom,False


#### Resample and Balance Data

Need to make sure that data is not imbalanced and introducing bias to the ML model. 
We will look at distribution of the target variable and resample and balance as needed. We are aiming to have 10,000 instances or less.

In [9]:
order_data['isCancellation'].value_counts()

False    527793
True       8704
Name: isCancellation, dtype: int64

We see that there is an imbalance in values. There are more orders than cancellations. In order to balance the number of samples for cancelled and not cancelled orders, we will resample the data to 50/50 after separating the two instances.

In [20]:
orders = order_data[~order_data['isCancellation']]
cancels = order_data[order_data['isCancellation']]

We will under-sample the larger dataset (orders) to match the length of the smaller dataset (cancels)

In [22]:
orders = orders.sample(n=len(cancels), random_state=12345)

We will then combine the two datasets again and reevaluate the instances

In [23]:
order_data = pd.concat([orders, cancels], axis=0)

In [24]:
len(order_data)

17408

We have more than 10,000 rows of data. We will then reduce the data by 50% through random sampling

In [26]:
order_data = order_data.sample(frac=0.5, random_state=12345)

In [27]:
len(order_data)

8704

In [30]:
order_data['isCancellation'].value_counts()

True     4404
False    4300
Name: isCancellation, dtype: int64

Checking our new dataset, we see that there is a fairly even distribution in the target variable and the dataset is below 10,000.

#### Drop Features that Aren't Useful for our Model

Now we will evaluate the features in our dataset for usability and relevance to our ML model. We will drop any variables that are not useful.

In [33]:
order_data.columns

Index(['StockCode', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country',
       'isCancellation'],
      dtype='object')

InvoiceNo, Description, Quantity, ImputedCustomerID will be removed from the dataset. 

InvoiceNo is unique for each order and therefore will not provide any meaningful insights into whether an order will be cancelled or not.
Description is the text correspondent of the StockCode and will therefore not provide extra value. 
Quantity varies for each order and contains some bias. Orders may be partial returns and will have a negative number. There is no way to differentiate new orders from partial returns. Therefore, we will drop it.
ImputedCustomerID is the same as the CustomerID so do not add value to the dataset. 


In [32]:
order_data.drop(['InvoiceNo', 'Description', 'Quantity', 'ImputedCustomerID'], axis=1, inplace=True)

In [34]:
order_data.columns

Index(['StockCode', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country',
       'isCancellation'],
      dtype='object')

#### Separate Data into Features and Labels

We will keep all independent variables/features that are not IsCancelled in the X df
We will keep only the target/dependent variable IsCancelled in the y df

In [35]:
X,y = order_data.iloc[:, :-1], order_data.iloc[:, -1]

In [36]:
X.head()

,StockCode,InvoiceDate,UnitPrice,CustomerID,Country
52732,22617,2011-02-07 10:44:00,4.95,16686.0,United Kingdom
301657,23370,2011-10-19 13:23:00,1.06,14306.0,United Kingdom
73782,21843,2011-03-07 12:06:00,10.95,17139.0,United Kingdom
352619,22739,2011-11-15 14:48:00,1.65,15720.0,United Kingdom
135427,23172,2011-05-17 11:47:00,1.65,17677.0,United Kingdom


In [37]:
y.head()

52732      True
301657    False
73782      True
352619    False
135427     True
Name: isCancellation, dtype: bool

#### Convert the InvoiceDate Feature to Month, Day, and Hour Features

When we look at the InvoiceDate variable, we see that is a string type object. We would like to separate this variable into Month, Day, and Hour variables for the model to use.

In [38]:
X['InvoiceDate'].info()

<class 'pandas.core.series.Series'>
Int64Index: 8704 entries, 52732 to 308688
Series name: InvoiceDate
Non-Null Count  Dtype 
--------------  ----- 
8704 non-null   object
dtypes: object(1)
memory usage: 136.0+ KB


We will first convert the string type to a datetime type in order to extract the other fields from the variable.

In [39]:
X['InvoiceDate'] = pd.to_datetime(X['InvoiceDate'], format='%Y-%m-%d %H:%M:%S')

Next, we will create Month, Day, Hour fields in the X dataframe, populate the appropriate fields by pulling from the InvoiceDate field, and drop the InvoiceDate field. 

In [40]:
X['Month'] = X['InvoiceDate'].dt.month
X['Day'] = X['InvoiceDate'].dt.day
X['Hour'] = X['InvoiceDate'].dt.hour 

In [41]:
X.drop('InvoiceDate', axis=1, inplace=True)

In [42]:
X.head()

,StockCode,UnitPrice,CustomerID,Country,Month,Day,Hour
52732,22617,4.95,16686.0,United Kingdom,2,7,10
301657,23370,1.06,14306.0,United Kingdom,10,19,13
73782,21843,10.95,17139.0,United Kingdom,3,7,12
352619,22739,1.65,15720.0,United Kingdom,11,15,14
135427,23172,1.65,17677.0,United Kingdom,5,17,11


#### One-Hot Encode the Country and StockCode Features

The Machine Learning model that we want to use to predict cancellations can only utilize numerical features. 

##### Country Variable
The Country variable is a text feature with various countries. 
In order to use these values, we will one-hot encode it so that every country has a column where its value is 1 to indicate the country associated with the order.

In [43]:
X['Country'].value_counts()

United Kingdom     7639
Germany             309
EIRE                212
France              146
USA                  57
Australia            48
Spain                37
Switzerland          34
Belgium              29
Italy                27
Japan                23
Portugal             20
Netherlands          17
Norway               13
Poland               13
Finland               9
Channel Islands       9
Malta                 8
Sweden                8
Austria               8
Cyprus                8
Denmark               6
Israel                5
Hong Kong             4
Unspecified           3
Canada                3
Singapore             3
Lebanon               1
Iceland               1
Lithuania             1
Bahrain               1
Czech Republic        1
Greece                1
Name: Country, dtype: int64

In [45]:
# One hot encode Country and drop the Country column after

dummies = pd.get_dummies(X['Country'])

In [47]:
dummies.head()

,Australia,Austria,Bahrain,Belgium,Canada,Channel Islands,Cyprus,Czech Republic,Denmark,EIRE,...,Norway,Poland,Portugal,Singapore,Spain,Sweden,Switzerland,USA,United Kingdom,Unspecified
52732,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
301657,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
73782,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
352619,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
135427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [48]:
# merge X and dummies dataframes and drop the country feature

X = pd.concat([X, dummies], axis=1).drop('Country', axis=1)

In [49]:
X.head()

,StockCode,UnitPrice,CustomerID,Month,Day,Hour,Australia,Austria,Bahrain,Belgium,...,Norway,Poland,Portugal,Singapore,Spain,Sweden,Switzerland,USA,United Kingdom,Unspecified
52732,22617,4.95,16686.0,2,7,10,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
301657,23370,1.06,14306.0,10,19,13,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
73782,21843,10.95,17139.0,3,7,12,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
352619,22739,1.65,15720.0,11,15,14,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
135427,23172,1.65,17677.0,5,17,11,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


##### StockCode

Similarly, while StockCode is in numerical form, the feature indicates a particular product and is therefore a categorical variable. We will make it more useable by converting it into one-hot encoded variables. 

In [51]:
dummy = pd.get_dummies(X['StockCode'])

In [52]:
dummy.head()

,10002,10125,10133,10135,11001,15034,15036,15039,15044A,15044B,...,90206C,90208,90209B,90209C,90210A,90211B,90214A,90214L,DCGS0004,DCGS0076
52732,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
301657,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73782,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352619,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We will add these columns to the X dataframe and drop the original StockCode column

In [53]:
X = pd.concat([X, dummy], axis=1).drop('StockCode', axis=1)

In [55]:
X.head()

,UnitPrice,CustomerID,Month,Day,Hour,Australia,Austria,Bahrain,Belgium,Canada,...,90206C,90208,90209B,90209C,90210A,90211B,90214A,90214L,DCGS0004,DCGS0076
52732,4.95,16686.0,2,7,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
301657,1.06,14306.0,10,19,13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73782,10.95,17139.0,3,7,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352619,1.65,15720.0,11,15,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
135427,1.65,17677.0,5,17,11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Model Training and Validation

The data is now ready to be fed into a machine learning model to train and validate. We will first create a DMatrix from the features (X) and labels (y), define model parameters and hyperparamters, and then train the model.

In [56]:
orders_dmatrix = xgb.DMatrix(data=X, label=y)

In [57]:
# Parameters for the XGBoost model
params = {'objective': "binary:logistic", "max_depth":3}

In [58]:
# Use cv function in xgb to train the model and perform k-fold cross validation on its predictions.

cv_result = xgb.cv(dtrain=orders_dmatrix, params=params, nfold=3, num_boost_round=5, metrics="error", as_pandas=True, seed=12345)

### Model Performance and Accuracy

Now that the model has been trained and cross-validated, we want to see how accurate it was at predicting. 
We will extract the test set's mean error from XGBoost's cross-validation results then compute the accuracy of the model as (1-error). 

In [59]:
print(cv_result)

   train-error-mean  train-error-std  test-error-mean  test-error-std
0          0.344439         0.003394         0.344785        0.006774
1          0.338235         0.002051         0.344785        0.007525
2          0.329906         0.002056         0.330538        0.001542
3          0.330997         0.005908         0.338007        0.011880
4          0.326517         0.000358         0.332836        0.002855


In [63]:
print("Accuracy:")
print(((1-cv_result['test-error-mean']).iloc[-1]))

Accuracy:
0.6671642097162562


The model is able to accurately predict whether an order will be cancelled or not 66.7% of the time. This is only slightly better than randomly guessing (50% accuracy). We may want to tune our model to get a better predictor score. 